# Aim

### What is Deposit

A deposit is a financial term that means money held at a bank. A deposit is a transaction involving a transfer of money to another party for safekeeping. However, a deposit can refer to a portion of money used as security or 
collateral for the delivery of a good.

<div class="alert alert-info"><strong>How a Deposit Works:</strong>
A deposit encompasses two different meanings. One kind of deposit involves a transfer of funds to another party for safekeeping. Using this definition, deposit refers to the money an investor transfers into a savings or checking account held at a bank or credit union.</div>

**In this case, we are going to analyze and predict which customer has deposit or liklyhood to put deposit**

# Libraries

In [ ]:
import os
import warnings
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import pickle
import plotly.graph_objects as go
import plotly.express as px 
from sklearn.svm import SVC
from sklearn import tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
pd.set_option('display.max_columns',None)
warnings.filterwarnings('ignore')

## Read The Data

`bank.csv` bank Data anlysis

Header | Definition
---|---------
`Age`| Age of customer
`Job` | Job of customer
`Martial` | Martial status of customer  
`Education` |Customer education level
`Default` |  Has credit in default?
`Housing` | If costumer has housing loan
`Loan` | Has Personal Loan
`Balance` |Customer's individual balance
`Contact` | Communication type
`Month` |  Last contact month of year 
`Day` | Last contact day of the week
`Duration` |Last contact duration, in seconds
`Campaign` | Number of contacts performed during this campaign and for this client
`Pdays` | Number of days that passed by after the client was last contacted from a previous campaign
`Previous` | Number of contacts performed before this campaign and for this client
`Poutcome` |outcome of the previous marketing campaign 
`Deposit` | has the client subscribed a term deposit

In [ ]:
df = pd.read_csv('../input/bank-marketing-dataset/bank.csv')

# EDA



<div class="alert alert-info"><strong>CLASS :  </strong>
      Let's try a new method instead of traditional analysis. We can create a new class for this. The purpose of the class is to analyze all the data and as a result. Class will understand which data type the column belongs to and visualize accordingly. While it will distribute data for columns whose data type is integer and automatically show us the mean, standard deviation, and minimum values. When there is categorical data, it will visually show us how much from which category.</div>

In [ ]:
class EDA:
    
    def row(self,data):
        fig = make_subplots(rows=1, cols=2)
        fig.add_trace(go.Indicator(mode = "number", value = data.shape[0], number={'font':{'color': '#E58F65','size':100}}, title = {"text": "🧾 Rows<br><span style='font-size:0.8em;color:gray'>In the Dataframe</span>"}, domain = {'x': [0, 0.5], 'y': [0.6, 1]}))
        fig.add_trace(go.Indicator(mode = "number", value = data.shape[1], number={'font':{'color': '#E58F65','size':100}}, title = {"text": "⭕ Columns<br><span style='font-size:0.8em;color:gray'>In the Dataframe</span>"}, domain = {'x': [0.5, 1], 'y': [0, 0.4]}))
        fig.show()
    
    def border_msg(self,msg, indent=1, width=None, title=None):
        """Print message-box with optional title."""
        lines = msg.split('\n')
        space = " " * indent
        if not width:
            width = max(map(len, lines))
        box = f'╔{"═" * (width + indent * 2)}╗\n'  
        if title:
            box += f'║{space}{title:<{width}}{space}║\n'  
            box += f'║{space}{"-" * len(title):<{width}}{space}║\n'  
        box += ''.join([f'║{space}{line:<{width}}{space}║\n' for line in lines])
        box += f'╚{"═" * (width + indent * 2)}╝' 
        print('\033[92m'+'\033[1m')
        print(box)
        
    def distribution(self,x,title):
        plt.figure(figsize=(10,8))
        ax = sns.distplot(x, kde=False,bins=30)
        values = np.array([rec.get_height() for rec in ax.patches])
        norm = plt.Normalize(values.min(), values.max())
        colors = plt.cm.jet(norm(values))
        for rec, col in zip(ax.patches,colors):
            rec.set_color(col)
        plt.title(title, size=20, color='black')
        
    def run(self,df):
        
        self.row(df)
        if len(df)>0:
            
            object_df = df.select_dtypes('object').columns.tolist()
            int_df = df.select_dtypes('int').columns.tolist()
            bool_df = df.select_dtypes('bool').columns.tolist()
            float_df = df.select_dtypes('float').columns.tolist()

            if len(object_df)>0:
                
                print( '\033[1m'+"OBJECT TYPE")
                for col in object_df:
                    self.border_msg(' '*25+ col.upper() + ' '*25)
                    self.border_msg('There are {} unique values in {} column'.format(df[col].nunique(),col.upper()))
                    plt.figure(figsize=(10,5))
                    sns.countplot(y = col, data = df,
                                  order = df[col].value_counts().index)
                    plt.show()
                    
            if len(int_df)>0:
                
                print('\033[1m'+"INT TYPE")
                for col in int_df:
                    self.border_msg(' '*25+ col.upper() + ' '*25)
                    self.border_msg('Average value is : {}'.format(df[col].mean()))
                    self.border_msg('Minumum value is : {}'.format(df[col].min()))
                    self.border_msg('Maximum value is : {}'.format(df[col].max()))
                    self.distribution(df[col],title=col)
                    if df[col].mean()>df[col].std():
                        print(self.border_msg("Normal distributed Data Located below mean"))
                        
                    elif df[col].mean()<df[col].std():
                        print(self.border_msg("Normal distributed Data Located above mean"))
                    else:
                        self.border_msg("Mean Equals Std Dev - Distribution is normal")
                        
                    fig = make_subplots(rows=1, cols=2)
                    fig.add_trace(go.Indicator(mode = "number", value = df[col].mean(), number={'font':{'color': '#E58F65','size':100}}, title = {"text": "📌 Mean<br><span style='font-size:0.8em;color:gray'></span>"}, domain = {'x': [0, 0.5], 'y': [0.6, 1]}))
                    fig.add_trace(go.Indicator(mode = "number", value = df[col].std(), number={'font':{'color': '#E58F65','size':100}}, title = {"text": "🖇 Standart dev<br><span style='font-size:0.8em;color:gray'></span>"}, domain = {'x': [0.5, 1], 'y': [0, 0.4]}))
                    fig.show()
                    plt.show()
                 

            if len(bool_df)>0:
                
                print('\033[1m'+"BOOL TYPE")
                for col in bool_df:
                    self.border_msg(' '*25+ col.upper() + ' '*25)
                    plt.figure(figsize=(10,5))
                    sns.countplot(y = col, data = df,
                                  order = df[col].value_counts().index)
                    plt.show()
                    
            if len(float_df)>0:
                
                print('\033[1m'+"FLOAT TYPE")
                for col in float_df:
                    for col in int_df:
                        self.distribution(df[col],title=col)
                        if df[col].mean()>df[col].std():
                            print(self.border_msg("Normal distributed Data Located below mean"))
                        
                        elif df[col].mean()<df[col].std():
                            print(self.border_msg("Normal distributed Data Located above mean"))
                        else:
                            self.border_msg("Mean Equals Std Dev - Distribution is normal")





In [ ]:
frame = EDA().run(df)

In [ ]:
sns.pairplot(df,hue='deposit',corner=True)

In [ ]:
plt.figure(figsize=(10,8),dpi=80)
sns.heatmap(df.corr(),cmap="coolwarm",annot=True,linewidth=0.5)
plt.yticks(rotation=55)

### Which occupation has most active balance?

In [ ]:
sample  = df.rename(columns={"balance":"Active Balance","job":"Occupation"})
fig = px.treemap(sample, path=[px.Constant('Active Balance'),'Occupation'], values='Active Balance',
                   hover_data=['Occupation'])
fig.show()

### As we can see Management more liklyhood to put deposit.

##### So, lets explore more information 

### We agree that most active balance and most deposit account mostly are managemnet. Does martial status has impact on deposit ?

In [ ]:
management = df[df.job=='management']
fig = px.box(management, x="marital", y="balance", 
             color="deposit", points="all",
            )
fig.update_traces(quartilemethod="exclusive")
fig.update_layout(
    title={
         'text': "Management behevior on deposit",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
    xaxis_title="Martial Status",
    yaxis_title="Active Balance",
    legend_title="Deposit Status",
    font=dict(
        family="Arial",
        size=18,
        color="DarkBlue"
    )
)
fig.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
fig.show()


In [ ]:
technician = df[df.job=='technician']
fig = px.box(technician, x="marital", y="balance", 
             color="deposit", points="all",
            )
fig.update_traces(quartilemethod="exclusive")
fig.update_layout(
    title={
         'text': "Technician behevior on deposit",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
    xaxis_title="Martial Status",
    yaxis_title="Active Balance",
    legend_title="Deposit Status",
    font=dict(
        family="Arial",
        size=18,
        color="DarkBlue"
    )
)
fig.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
fig.show()


In [ ]:
blue = df[df.job=='blue-collar']
fig = px.box(blue, x="marital", y="balance", 
             color="deposit", points="all",
            )
fig.update_traces(quartilemethod="exclusive")
fig.update_layout(
    title={
         'text': "Blue-Collar behevior on deposit",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
    xaxis_title="Martial Status",
    yaxis_title="Active Balance",
    legend_title="Deposit Status",
    font=dict(
        family="Arial",
        size=20,
        color="DarkBlue"
    )
)
fig.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
fig.show()


### Does Education and Martial Status related on deposit ?

*According to educational status, marital status somehow affected active balance.*

In [ ]:
education= df.groupby(['marital','education'], as_index=False)['balance'].agg({np.median,np.mean})
education = round(education,2)

ax = education.unstack(level=0).plot(kind='bar', subplots=True, rot=45, figsize=(20, 8), layout=(1, 6))
plt.tight_layout()

## What could be the reason low balance? does anything effect balance

In [ ]:
fig = px.scatter(df, x="balance", y="duration", color="marital", 
                 marginal_x="box", marginal_y="violin",
                  title="Click on the legend items!")
fig.update_layout(
    title={
         'text': "Purpose Of Low Amount Of Balance.",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
    xaxis_title="Duration",
    yaxis_title=" Balance",
    legend_title="Marital Status",
    font=dict(
        family="Arial",
        size=20,
        color="DarkBlue"
    )
)
fig.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
fig.show()

* Marital Status: As discussed previously, the impact of a divorce has a significant impact on the balance of the individual.
* Education: The level of education also has a significant impact on the amount of balance a prospect has.
* Loans: Whether the prospect has a previous loan has a significant impact on the amount of balance he or she has.**

# Target 

### It is really good that we have almost same amount of data for each class.

In [ ]:
sns.countplot(data=df,x='deposit')

In [ ]:
df.deposit.value_counts()

>Before we split the data we need to encode categorical column.
> For this purpose we are going to use 2 different method
> * Label Encoder
> * One-Hot Encoder

In [ ]:
df.head()

# Label Encoder


 What is a label encoder?
 Image result for label encoder
Label Encoding is a popular encoding technique for handling categorical variables. In this technique, each label is assigned a unique integer based on alphabetical ordering

In [ ]:
le = LabelEncoder()
df.marital = le.fit_transform(df.marital)
df.housing = le.fit_transform(df.housing)
df.deposit = le.fit_transform(df.deposit)
df.loan = le.fit_transform(df.loan)
df.default = le.fit_transform(df.default)

# One-Hot Encoder

One-Hot Encoding is another popular technique for treating categorical variables. It simply creates additional features based on the number of unique values in the categorical feature. Every unique value in the category will be added as a feature.

In [ ]:
def get_encoder_inst(feature_col):
  
    assert isinstance(feature_col, pd.Series)
    feature_vec = feature_col.sort_values().values.reshape(-1, 1)
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(feature_vec) 
  
    filename = '.pickle'
    pickle.dump(enc, open(filename, 'wb'))
    return enc

def get_one_hot_enc(feature_col, enc,cols):
  
    assert isinstance(feature_col, pd.Series)
    assert isinstance(enc, OneHotEncoder)
    unseen_vec = feature_col.values.reshape(-1, 1)
    encoded_vec = enc.transform(unseen_vec).toarray()
    column_name = enc.get_feature_names([cols])
    encoded_df = pd.DataFrame(encoded_vec, columns= column_name)
    return encoded_df

### *Lets create categorical list*

In [ ]:
ohe_cat_list = ['job','education','month','contact','poutcome']
ohe_cat_data = df[ohe_cat_list]
df.drop(ohe_cat_list,axis=1,inplace=True)

In [ ]:
data_list = []
for cols in ohe_cat_data.columns:
    encoder = get_encoder_inst(ohe_cat_data[cols])
    one = get_one_hot_enc(ohe_cat_data[cols],encoder,cols)
    data_list.append(one)
    
final_ohe = pd.concat(data_list,axis=1)
df.reset_index(drop=True, inplace=True)
final_ohe.reset_index(drop=True, inplace=True)
for cols in final_ohe.columns:
    final_ohe[cols] = final_ohe[cols].astype('int')

In [ ]:
df = pd.concat([df,final_ohe],axis=1)

# Split Data

In [ ]:
X = df.drop('deposit',axis=1)
y = df[['deposit']]

# K-Fold

In [ ]:
KF = KFold(n_splits=3,shuffle=True)
for train_index, test_index in KF.split(X):
    x_train, x_test = X.iloc[list(train_index)], X.iloc[list(test_index)]
    y_train, y_test = y.iloc[list(train_index)], y.iloc[list(test_index)]

In [ ]:
print("Train data shape:{}".format(x_train.shape))
print("Test data shape:{}".format(x_test.shape))

# Scaling Data

In [ ]:
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(x_train)
scaled_test = scaler.transform(x_test)


#### *Everything are all set up*


# Model

### The Decision-Tree algorithm is one of the most frequently and widely used supervised machine learning algorithms that can be used for both classification and regression tasks.
#### The intuition behind the Decision-Tree algorithm is very simple to understand.
##### The Decision Tree algorithm intuition is as follows:-

* For each attribute in the dataset, the Decision-Tree algorithm forms a node. The most important attribute is placed at the root node.

* For evaluating the task in hand, we start at the root node and we work our way down the tree by following the corresponding node that meets our condition or decision.

* This process continues until a leaf node is reached. It contains the prediction or the outcome of the Decision Tree.

In [ ]:
dt = DecisionTreeClassifier()

In [ ]:
dt.fit(scaled_train,y_train)

In [ ]:
pred = dt.predict(scaled_test)

In [ ]:
accuracy_score(y_test,pred)

In [ ]:
param_grid = {'criterion': ['gini', 'entropy'],  #scoring methodology; two supported formulas for calculating information gain - default is gini
              'splitter': ['best', 'random'], #splitting methodology; two supported strategies - default is best
              'max_depth': [2,4,6,8,10,None], #max depth tree can grow; default is none
              'min_samples_split': [2,5,10,.03,.05], #minimum subset size BEFORE new split (fraction is % of total); default is 2
              'min_samples_leaf': [1,5,10,.03,.05], #minimum subset size AFTER new split split (fraction is % of total); default is 1
              'max_features': [None, 'auto'], #max features to consider when performing split; default none or all
              'random_state': [0] #seed or control random number generator
             }

In [ ]:
tune_model = GridSearchCV(DecisionTreeClassifier(), 
                          param_grid=param_grid, 
                          scoring = 'roc_auc',
                          cv = 5,
                          verbose=0)
tune_model.fit(scaled_train, y_train)

In [ ]:
print('\033[1m'+'Decision Tree Parameters:{} '.format(tune_model.best_params_))

In [ ]:
dt_tuned =  DecisionTreeClassifier(criterion='gini',
                                   min_samples_split=0.03,
                                   max_depth=None,
                                    max_features = None,
                                   min_samples_leaf=10,
                                   random_state = 0,
                                   splitter='random')

dt_tuned.fit(scaled_train,y_train)

In [ ]:
pred = dt_tuned.predict(scaled_test)

In [ ]:
accuracy_score(y_test,pred)

In [ ]:
print(classification_report(y_test,pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

plot_confusion_matrix(dt_tuned,scaled_test,y_test,ax=ax)

In [ ]:
tree_model = dt_tuned.fit(scaled_train, y_train)
importances = tree_model.feature_importances_
feature_names = df.drop('deposit', axis=1).columns
indices = np.argsort(importances)[::-1]

# Plot the feature importances of the forest
def feature_importance_graph(indices, importances, feature_names):
    plt.figure(figsize=(15,15))
    plt.title(" Feature importances \n with DecisionTreeClassifier", fontsize=18)
    plt.barh(range(len(indices)), importances[indices], color='#fffb00',  align="center")
    plt.yticks(range(len(indices)), feature_names[indices], rotation='horizontal',fontsize=14)
    plt.ylim([-1, len(indices)])

feature_importance_graph(indices, importances, feature_names)
plt.show()

# Determine Best Trashold 

We are going to use pericision and recall tradeoff technique. 
#### How can we decide which threshold to use? We want to return the scores instead of predictions with this code.

In [ ]:
y_scores = dt_tuned.predict_proba(scaled_train)

if y_scores.ndim == 2:
    y_scores = y_scores[:, 1]
    
precisions, recalls, threshold = precision_recall_curve(y_train, y_scores,)

In [ ]:
def precision_recall_curve(precisions, recalls, thresholds):
    fig, ax = plt.subplots(figsize=(12,8))
    plt.plot(thresholds, precisions[:-1], "g--", label="Precisions")
    plt.plot(thresholds, recalls[:-1], "#1e81b0", label="Recalls")
    plt.title("Precision and Recall \n Tradeoff", fontsize=18)
    plt.ylabel("Level of Precision and Recall", fontsize=16)
    plt.xlabel("Thresholds", fontsize=16)
    plt.legend(loc="best", fontsize=14)
    plt.xlim([-2, 4.7])
    plt.ylim([0, 1])
    plt.axvline(x=0.58, linewidth=3, color="#0B3861")
    plt.annotate('Best Precision and \n Recall Balance \n is at 0.58 \n threshold ', xy=(0.62, 0.80), xytext=(55, -40),
             textcoords="offset points",
            arrowprops=dict(facecolor='black', shrink=0.5),
                fontsize=12, 
                color='k')
    plt.show()
    
precision_recall_curve(precisions, recalls,threshold)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.title("Precision and Recall Curve", fontsize=18)
plot_precision_recall_curve(dt_tuned,scaled_test,y_test,ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.title("ROC Curve", fontsize=18)
plot_roc_curve(dt_tuned,scaled_test,y_test,ax=ax)

## Print Text Representation
Exporting Decision Tree to the text representation can be useful when working on applications whitout user interface or when we want to log information about the model into the text file.

In [ ]:
text_representation = tree.export_text(tree_model)
print('\033[1m'+'\033[92m'+text_representation)

# Tree Plot 
### The sample counts that are shown are weighted with any sample_weights that might be present.
##### The visualization is fit automatically to the size of the axis. Use the figsize or dpi arguments of plt.figure to control the size of the rendering.
Tree to large to analyze. `click` or `save` the plot and see the nodes and leaf. It shows  how decision made by model

In [ ]:
fig = plt.figure(figsize=(150,150))
_ = tree.plot_tree(dt_tuned, feature_names=feature_names,
                    class_names= ['yes','no'],
                   filled=True)

# 📔Additional Notes

###  Months of Marketing Activity💸:
We saw that the the month of highest level of marketing activity was the month of May. However, this was the month that potential clients tended to reject term deposits offers (Lowest effective rate: -34.49%). For the next marketing campaign, it will be wise for the bank to focus the marketing campaign during the months of March, September, October and December. (December should be under consideration because it was the month with the lowest marketing activity, there might be a reason why december is the lowest.)

### 📌Seasonality: 
Potential clients opted to suscribe term deposits during the seasons of fall and winter. The next marketing campaign should focus its activity throghout these seasons.

### 📌Campaign Calls:
A policy should be implemented that states that no more than 3 calls should be applied to the same potential client in order to save time and effort in getting new potential clients. Remember, the more we call the same potential client, the likely he or she will decline to open a term deposit.

### 📌Age Category: 
The next marketing campaign of the bank should target potential clients in their 20s or younger and 60s or older. The youngest category had a 60% chance of suscribing to a term deposit while the eldest category had a 76% chance of suscribing to a term deposit. It will be great if for the next campaign the bank addressed these two categories and therefore, increase the likelihood of more term deposits suscriptions.

###  📌Occupation:
Not surprisingly, potential clients that were students or retired were the most likely to suscribe to a term deposit. Retired individuals, tend to have more term deposits in order to gain some cash through interest payments. Remember, term deposits are short-term loans in which the individual (in this case the retired person) agrees not to withdraw the cash from the bank until a certain date agreed between the individual and the financial institution. After that time the individual gets its capital back and its interest made on the loan. Retired individuals tend to not spend bigly its cash so they are morelikely to put their cash to work by lending it to the financial institution. Students were the other group that used to suscribe term deposits.

### 📌House Loans and Balances:
Potential clients in the low balance and no balance category were more likely to have a house loan than people in the average and high balance category. What does it mean to have a house loan? This means that the potential client has financial compromises to pay back its house loan and thus, there is no cash for he or she to suscribe to a term deposit account. However, we see that potential clients in the average and hih balances are less likely to have a house loan and therefore, more likely to open a term deposit. Lastly, the next marketing campaign should focus on individuals of average and high balances in order to increase the likelihood of suscribing to a term deposit.


### 📌Develop a Questionaire during the Calls: 
Since duration of the call is the feature that most positively correlates with whether a potential client will open a term deposit or not, by providing an interesting questionaire for potential clients during the calls the conversation length might increase. Of course, this does not assure us that the potential client will suscribe to a term deposit! Nevertheless, we don't loose anything by implementing a strategy that will increase the level of engagement of the potential client leading to an increase probability of suscribing to a term deposit, and therefore an increase in effectiveness for the next marketing campaign the bank will excecute.


### 📌Target individuals with a higher duration (above 375):
Target the target group that is above average in duration, there is a highly likelihood that this target group would open a term deposit account. The likelihood that this group would open a term deposit account is at 78% which is pretty high. This would allow that the success rate of the next marketing campaign would be highly successful.


# END

* Please Upvote if you like 👍
* Let's discuss any idea about this topic on comments

##### Resource
* https://www.tutorialspoint.com/bank_management/bank_management_marketing.htm#:~:text=In%20simple%20words%2C%20bank%20marketing,the%20bank%20and%20environmental%20constraints.
* https://www.kaggle.com/janiobachmann/bank-marketing-campaign-opening-a-term-deposit